# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [2]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np

def generate_energy_df():
    # load dataframe and remove header and footer
    energy_indicators_df = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38, usecols={2,3,4,5}, names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], na_values='...')
    
    # YOUR CODE HERE transform data
    energy_indicators_df['Energy Supply'] *= 1000000
    # YOUR CODE HERE transform Country column
    country_renames = {'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong'}
    
    energy_indicators_df['Country'] = energy_indicators_df['Country'].replace(country_renames)

    # Remove digits from country names
    energy_indicators_df['Country'] = energy_indicators_df['Country'].str.replace(r'\(.*\)', '', regex=True).str.strip()
    energy_indicators_df['Country'] = energy_indicators_df['Country'].str.replace(r'\d+', '', regex=True).str.strip()
    
    print('Energy DataFrame List: \n', energy_indicators_df['Country'].unique())
    
    return energy_indicators_df

def get_GDP_df():
    def is_number(s: str) -> bool:
        """
        Check if a string is a number.

        Args:
            s (str): The string to check.

        Returns:
            A boolean indicating if the string is a number.
        """
        try:
            int(s)  
            return True
        except ValueError:
            return False

    # load data, remove header, adjust column names
    
    gdp_df = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    gdp_country_renames = {'Korea, Rep.': 'South Korea', 
    'Iran, Islamic Rep.': 'Iran',
    'Hong Kong SAR, China': 'Hong Kong'}
    
    gdp_df['Country Name'] = gdp_df['Country Name'].replace(gdp_country_renames)

    # transform Country column 
    
    gdp_df = gdp_df[['Country Name'] + [str(year) for year in range(2006, 2016)]]
    gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)
    
    
    return gdp_df

def get_scimago_df():
    #  load data
    
    scimago_df = pd.read_excel('assets/scimagojr-3.xlsx')
    
    
    return scimago_df

def answer_one():
    energy_indicators_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()
    
# merge
    merged_df = pd.merge(scimago_df, energy_indicators_df, how='inner', left_on='Country', right_on='Country')
    merged_df = pd.merge(merged_df, gdp_df, how='inner', left_on='Country', right_on='Country')
    
    

    merged_df = merged_df[merged_df['Rank'] <= 15]

# Index of Country
    merged_df.set_index('Country', inplace=True)

# Select and order the columns
    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + [str(year) for year in range(2006, 2016)]
    merged_df = merged_df[columns]

    return merged_df

answer_one().head

Energy DataFrame List: 
 ['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia' 'Bonaire, Sint Eustatius and Saba' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'British Virgin Islands' 'Brunei Darussalam'
 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon'
 'Canada' 'Cayman Islands' 'Central African Republic' 'Chad' 'Chile'
 'China' 'China, Hong Kong Special Administrative Region'
 'China, Macao Special Administrative Region' 'Colombia' 'Comoros' 'Congo'
 'Cook Islands' 'Costa Rica' "Côte d'Ivoire" 'Croatia' 'Cuba' 'Curaçao'
 'Cyprus' 'Czech Republic' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 

<bound method NDFrame.head of                     Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     127050             126767     597237   
Japan                  3      30504              30287     223024   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       8896               8819      57470   
Australia             14       8831               8725      90765   
Braz

In [24]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

Energy DataFrame List: 
 ['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia' 'Bonaire, Sint Eustatius and Saba' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'British Virgin Islands' 'Brunei Darussalam'
 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon'
 'Canada' 'Cayman Islands' 'Central African Republic' 'Chad' 'Chile'
 'China' 'China, Hong Kong Special Administrative Region'
 'China, Macao Special Administrative Region' 'Colombia' 'Comoros' 'Congo'
 'Cook Islands' 'Costa Rica' "Côte d'Ivoire" 'Croatia' 'Cuba' 'Curaçao'
 'Cyprus' 'Czech Republic' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 

AssertionError: Q1: Your DataFrame should have 20 columns and 15 entries!

In [ ]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [19]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [5]:
def answer_two():
    energy_indicators_df = generate_energy_df()
    gpd_df = get_GDP_df()
    scimago_df = get_scimago_df()
    
    # Substract UNION entries (outer) with INTERSECT entries (default) to get lost entries quantity
    union_df = generate_energy_df.merge(scimago_df.merge(gpd_df, on='Country'), on='Country', how='outer')
    intersect_df = generate_energy_df.merge(scimago_df.merge(gpd_df, on='Country'), on='Country')
    
    lost_entries = union_df.shape[0] - intersect_df.shape[0]
    
    return lost_entries

answer_two().head()

Energy DataFrame List: 
 ['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia' 'Bonaire, Sint Eustatius and Saba' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'British Virgin Islands' 'Brunei Darussalam'
 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon'
 'Canada' 'Cayman Islands' 'Central African Republic' 'Chad' 'Chile'
 'China' 'China, Hong Kong Special Administrative Region'
 'China, Macao Special Administrative Region' 'Colombia' 'Comoros' 'Congo'
 'Cook Islands' 'Costa Rica' "Côte d'Ivoire" 'Croatia' 'Cuba' 'Curaçao'
 'Cyprus' 'Czech Republic' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 

AttributeError: 'function' object has no attribute 'merge'

In [ ]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [ ]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [ ]:
def answer_three():
    answer_1_df = answer_one()
    
    avg_gdp = answer_1_df['Country Name'] + [str(year) for year in range(2006, 2016)].mean(axis=1)
    avg_gdp = avg_gdp.sort_values(ascending=False)
    
    return avg_gdp

answer_three()

In [ ]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [ ]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [ ]:
def answer_four():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [ ]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [ ]:
def answer_five():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [ ]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [ ]:
def answer_six():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [ ]:
def answer_seven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [ ]:
def answer_eight():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [ ]:
def answer_nine():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [ ]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [ ]:
def answer_ten():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [ ]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [ ]:
def answer_eleven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [ ]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [ ]:
def answer_twelve():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"

In [ ]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [ ]:
def answer_thirteen():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [ ]:
# space for proffesor tests